# Getty Query Tutorial (Date of Birth)
In this notebook, we perform the similar hunger-for-knowledge procedure for date of birth in Wikidata and Getty Vocabulary. The main difference is that we generalize the query for single property to the property chain, this is, the path in kgtk.

In [1]:
import os
import json
import pandas as pd

from kgtk.functions import kgtk, kypher

## Step 0: Set up environment paths
Here we set needed Graph files and desired output files to environment paths.

In [2]:
# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

# Folder where database files store
data_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
os.environ['DATABASE'] = data_path
kgtk_environment_variables.append('DATABASE')

# Wikidata (all is much less than claims)
os.environ['WIKIDATA'] = data_path + "claims.tsv"
kgtk_environment_variables.append('WIKIDATA')

# Label file of Wikidata
os.environ['KGTK_LABEL_FILE'] = data_path + "labels.en.tsv"
kgtk_environment_variables.append('KGTK_LABEL_FILE')

# P31
os.environ['P31'] = data_path + "P31.tsv"
kgtk_environment_variables.append('P31')

# P279star
os.environ['P279STAR'] = data_path + "P279star.tsv"
kgtk_environment_variables.append('P279STAR')

# Folder of ULAN
ulan_path = data_path + "gvp/ULAN/"
os.environ['ULAN'] = ulan_path
kgtk_environment_variables.append('ULAN')

# Full file provided by Getty 
# Full file is much truncated in this process thus we will use concat files below instead.
# os.environ['ULAN_FULL'] = ulan_path + "full.tsv"
# kgtk_environment_variables.append('ULAN_FULL')

# File concatenated by used explicit files: ULAN Subjects, AgentMap, Biographies
# The suffix ID means we performed add-id operation to the graph to meet the requirement of paths command.
ulan_concat_path = ulan_path + "ulan.bio.concat.id.tsv"
os.environ['ULAN_BIO_CONCAT_ID'] = ulan_concat_path
kgtk_environment_variables.append('ULAN_BIO_CONCAT_ID')

# ULAN-Wikidata alignment file, maps ULAN ID to Wikidata Qnode
ulan_wikialign_path = ulan_path + "wiki.align.tsv"
os.environ['ULAN_ALIGN'] = ulan_wikialign_path
kgtk_environment_variables.append('ULAN_ALIGN')

# Output
output_path = data_path + "gvp/dob_output/"
if not os.path.exists(output_path):
    os.mkdir(output_path)
os.environ['OUTPUT'] = output_path
kgtk_environment_variables.append('OUTPUT')

# Each file will be explained later in the procedure
output_names = {
    "wiki_dob": "wiki.dob.tsv",
    "wiki_unknown": "wiki.unknown.tsv",
    "pairs": "pairs.tsv",
    "paths": "paths.tsv",
    "paths_label": "paths.label.tsv",
    "getty_dob": "getty.dob.tsv",
    "getty_mapped": "getty.mapped.tsv",
    "getty_agree": "getty.agree.tsv",
    "correct": "correct.tsv",
    "unknown": "unknown.tsv"
}

for key, value in output_names.items():
    variable = key.upper()
    os.environ[variable] = os.path.join(output_path, value)
    kgtk_environment_variables.append(variable)

for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

DATABASE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
WIKIDATA: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/claims.tsv"
KGTK_LABEL_FILE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/labels.en.tsv"
P31: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P31.tsv"
P279STAR: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P279star.tsv"
ULAN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/"
ULAN_BIO_CONCAT_ID: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/ulan.bio.concat.id.tsv"
ULAN_ALIGN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/wiki.align.tsv"
OUTPUT: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/dob_output/"
WIKI_DOB: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/dob_output/wiki.dob.tsv"
WIKI_UNKNOWN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/dob_output/wiki.unknown.tsv"
PAIRS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/dob_output/pairs.tsv"
PATHS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/g

## Step 1: Query Wikidata
Query Wikidata for PoB for people that have an ULAN ID (92k people)

In [3]:
# count total poeple
# ULAN ID
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_627_c1.""node1"")"
0,92353


In [4]:
# Qnode
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c1.""node2"")"
0,92210


In [ ]:
align_df = pd.read_csv(os.environ["ULAN_ALIGN"], sep='\t')
multiple = align_df.groupby('node1').agg({'node2': lambda x: list(x)})
multiple.loc[multiple['node2'].map(len) > 2]

In [5]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P569]->(dob)' 
        --return 'distinct ulanid as node1, "P569" as label, dob as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 73.4 ms, sys: 51.7 ms, total: 125 ms
Wall time: 1min 37s


,node1,label,node2,label;label
0,ulan:500023949,P569,^1886-03-24T00:00:00Z/11,'date of birth'@en
1,ulan:500123827,P569,^1884-04-03T00:00:00Z/11,'date of birth'@en
2,ulan:500123827,P569,^1885-04-03T00:00:00Z/11,'date of birth'@en
3,ulan:500223082,P569,^1944-01-27T00:00:00Z/11,'date of birth'@en
4,ulan:500108173,P569,^1951-07-26T00:00:00Z/11,'date of birth'@en
5,ulan:500072302,P569,^1890-06-18T00:00:00Z/11,'date of birth'@en
6,ulan:500091345,P569,^1885-11-22T00:00:00Z/11,'date of birth'@en
7,ulan:500059872,P569,^1953-06-07T00:00:00Z/11,'date of birth'@en
8,ulan:500020713,P569,^1881-03-20T00:00:00Z/11,'date of birth'@en
9,ulan:500160869,P569,^1903-11-15T00:00:00Z/11,'date of birth'@en


In [6]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P569]->(dob)' 
        --return 'distinct ulanid as node1, "P569" as label, dob as node2'
        -o $WIKI_DOB
    """)

CPU times: user 15.6 ms, sys: 16.2 ms, total: 31.8 ms
Wall time: 30.2 s


## Step 2: Record Known and Unknown

In [7]:
# count known ULAN IDs
kgtk("""
    query -i $WIKI_DOB
        --match '(ulanid)-[]->(dob)' 
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_660_c1.""node1"")"
0,84110


In [3]:
# count known Qnodes
kgtk("""
    query -i $WIKI_DOB $ULAN_ALIGN
        --match 'd: (ulanid)-[]->(nation), a: (ulanid)-[]->(qnode)' 
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c2.""node2"")"
0,83993


Filter unknow in ULAN-Wikidata alignment

In [8]:
%%time
kgtk("""
    ifnotexists -i $ULAN_ALIGN 
        --filter-on $WIKI_DOB 
        --input-keys node1 
        --filter-keys node1 
        -o $WIKI_UNKNOWN
    """)

CPU times: user 5.4 ms, sys: 9.94 ms, total: 15.3 ms
Wall time: 4.79 s


In [9]:
# count unknown ULAN IDs
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_661_c1.""node1"")"
0,8243


In [4]:
# count unknown Qnodes
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_661_c1.""node2"")"
0,8217


## Step 3: Query Getty
Based on the known PoB for people, query Getty. Before we perform single property query (we can consider it as 1 hop paths query), this time, we generalize to paths between known pairs (Subjects, Objects). Similarily we will compute the distributions of found paths and pick the top 1 path (property chain). 

### 3-1: Build Pairs from Wikidata Results 
Pairs file is needed for performing kgtk paths operation as the query objects.

In [25]:
%%time
kgtk("""
    query -i $WIKI_DOB
        --match '(ulanid)-[]->(dob)'
        --return 'distinct ulanid as source, kgtk_stringify(kgtk_date_year(dob)) as target'
        --limit 200
        -o $PAIRS
    """)

CPU times: user 4.13 ms, sys: 10.5 ms, total: 14.6 ms
Wall time: 6.2 s


In [26]:
!head $PAIRS | column -ts $'\t'

source          target
ulan:500023949  "1886"
ulan:500123827  "1884"
ulan:500123827  "1885"
ulan:500223082  "1944"
ulan:500108173  "1951"
ulan:500072302  "1890"
ulan:500091345  "1885"
ulan:500059872  "1953"
ulan:500020713  "1881"


### 3-2: Perform Paths Mapping
This step is the core step of this query and it is also the bottleneck of running this procedure.

In [27]:
%%time
kgtk("""
    paths --max_hops 5
        --path-file $PAIRS
        --path-mode NONE 
        --path-source source
        --path-target target
        -i $ULAN_BIO_CONCAT_ID
        --statistics-only
        -o $PATHS
    """)

CPU times: user 144 ms, sys: 57.3 ms, total: 201 ms
Wall time: 3min 32s


In [28]:
!head $PATHS

node1	label	node2	id
p0	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p0-0-0
p0	1	ulan:500023949-agent:gvp:biographyNonPreferred:ulan_bio:4000061262	p0-1-1
p0	2	ulan_bio:4000061262:gvp:estStart:nodeBKhLqNtUciRjAx7zs7VoBJ-236987	p0-2-2
p0	3	nodeBKhLqNtUciRjAx7zs7VoBJ-236987:gvp:structured_value:"1886"	p0-3-3
p1	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p1-0-4
p1	1	ulan:500023949-agent:gvp:biographyNonPreferred:ulan_bio:4000061263	p1-1-5
p1	2	ulan_bio:4000061263:gvp:estStart:nodeBKhLqNtUciRjAx7zs7VoBJ-236989	p1-2-6
p1	3	nodeBKhLqNtUciRjAx7zs7VoBJ-236989:gvp:structured_value:"1886"	p1-3-7
p2	0	ulan:500023949:foaf:focus:ulan:500023949-agent	p2-0-8


### 3-3: Select the Top 1 Property Chain
Here we use Pandas to sort the top 1 property chain. 

In [29]:
# load paths.tsv file
paths = pd.read_csv(os.environ['PATHS'], sep='\t')
# process property: since paths command generate id as node2, we need to further extract property from it
paths.node2 = paths.node2.apply(lambda x: ':'.join(x.split(':')[2:4]) if len(x.split(':')) > 4 else ':'.join(x.split(':')[1:3]))
# save processed property
paths.to_csv(os.environ['PATHS_LABEL'], sep='\t', index=False)
# join property chain for each found path
paths_concat = paths.groupby(paths['node1']).agg({'node2': lambda x: ' '.join(list(x))})
# sort the top 
paths_concat.value_counts()#.head(1)

node2                                                                 
foaf:focus gvp:biographyNonPreferred gvp:estStart gvp:structured_value    196
foaf:focus gvp:biographyPreferred gvp:estStart gvp:structured_value       153
dtype: int64

### 3-4: Query Getty Using the Property Chain
Query Getty for the values of the unknown people, with the found property chain: `(ulanid)-[foaf:focus]->()-[gvp:biographyNonPreferred]->()-[gvp:estStart]->()-[gvp:structured_value]->(String of Birth Year)`

In [3]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_BIO_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(dob)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "gvp:estStart" AND
                 p3.label = "gvp:structured_value"'
        --return 'distinct ulanid as node1, "P569" as label, printf("^%s-01-01T00:00:00Z/9", kgtk_unstringify(dob)) as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 42.2 ms, sys: 38.5 ms, total: 80.7 ms
Wall time: 1min 21s


,node1,label,node2,label;label
0,ulan:500000007,P569,^1516-01-01T00:00:00Z/9,'date of birth'@en
1,ulan:500000009,P569,^1540-01-01T00:00:00Z/9,'date of birth'@en
2,ulan:500000015,P569,^1754-01-01T00:00:00Z/9,'date of birth'@en
3,ulan:500000024,P569,^1446-01-01T00:00:00Z/9,'date of birth'@en
4,ulan:500000031,P569,^1371-01-01T00:00:00Z/9,'date of birth'@en
5,ulan:500000048,P569,^1788-01-01T00:00:00Z/9,'date of birth'@en
6,ulan:500000051,P569,^1774-01-01T00:00:00Z/9,'date of birth'@en
7,ulan:500000065,P569,^1271-01-01T00:00:00Z/9,'date of birth'@en
8,ulan:500000066,P569,^1664-01-01T00:00:00Z/9,'date of birth'@en
9,ulan:500000069,P569,^1271-01-01T00:00:00Z/9,'date of birth'@en


In [4]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_BIO_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(dob)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "gvp:estStart" AND
                 p3.label = "gvp:structured_value"'
        --return 'distinct ulanid as node1, "P569" as label, printf("^%s-01-01T00:00:00Z/9", kgtk_unstringify(dob)) as node2'
        -o $GETTY_DOB
    """)

CPU times: user 5.93 ms, sys: 7.87 ms, total: 13.8 ms
Wall time: 4.56 s


Count how many new results (count in ULAN ID) we found

In [5]:
kgtk("""
    query -i $GETTY_DOB
        --match '(ulanid)-[]->(dob)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_7168_c1.""node1"")"
0,8243


## Step 4: Record New Results

Map back to Wikidata

In [6]:
%%time
kgtk("""
    query -i $GETTY_DOB $ULAN_ALIGN
        --match 'g: (ulanid)-[p]->(dob), 
                 w: (ulanid)-[]->(qnode)'
        --return 'distinct qnode as node1, p.label as label, dob as node2'
        -o $GETTY_MAPPED
    """)

CPU times: user 6.37 ms, sys: 7.6 ms, total: 14 ms
Wall time: 3.31 s


Count how many new results (in Qnode) we found

In [7]:
kgtk("""
    query -i $GETTY_MAPPED
        --match '(qnode)-[]->(dob)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_7169_c1.""node1"")"
0,8217


## Step 5: Validate with Wikidata Constraints
Since

In [8]:
!kgtk validate -i $GETTY_DOB --ignore-minimum-year


Data lines read: 8243
Data lines passed: 8243


## Step 6: Record Unknown after Query
We record unknowns after query Getty.

In [9]:
kgtk("""
    ifnotexists -i $WIKI_UNKNOWN \
        --filter-on $GETTY_DOB \
        --input-keys node1 \
        --filter-keys node1
        -o $UNKNOWN
    """)

In [10]:
# count in ULAN IDs
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_7170_c1.""node1"")"
0,0


In [11]:
# count in Qnodes
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_7170_c1.""node2"")"
0,0


# Step 7: Comparison of the Known Values
How often do the results from two knowledge graphs agree

In [12]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $ULAN_BIO_CONCAT_ID 
        --match 'w: (ulanid)-[]->(qnode), 
                 u: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(dob)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "gvp:estStart" AND
                 p3.label = "gvp:structured_value"'
        --return 'distinct ulanid as node1, "P569" as label, dob as node2'
        -o $GETTY_AGREE
    """)

CPU times: user 5.36 ms, sys: 9.44 ms, total: 14.8 ms
Wall time: 6.35 s


Count agree in ULAN ID:

In [13]:
%%time
kgtk("""
    query -i $WIKI_DOB $GETTY_AGREE
        --match 'w: (ulanid)-[]->(dob), g: (ulanid)-[]->(yob)'
        --where 'kgtk_stringify(kgtk_date_year(dob)) = yob'
        --return 'count(distinct ulanid)'
    """)
# database is locked

CPU times: user 8.23 ms, sys: 11.4 ms, total: 19.6 ms
Wall time: 4.67 s


,"count(DISTINCT graph_660_c1.""node1"")"
0,70425


Count agree in Qnode:

In [14]:
%%time
kgtk("""
    query -i $WIKI_DOB $GETTY_AGREE $ULAN_ALIGN
        --match 'd: (ulanid)-[]->(dob), g: (ulanid)-[]->(yob), a: (ulanid)-[]->(qnode)'
        --where 'kgtk_stringify(kgtk_date_year(dob)) = yob'
        --return 'count(distinct qnode)'
    """)

CPU times: user 9.14 ms, sys: 9.35 ms, total: 18.5 ms
Wall time: 4.47 s


,"count(DISTINCT graph_627_c3.""node2"")"
0,70366


Count value per node:

In [15]:
nodes = kgtk("""
    query -i $GETTY_AGREE $ULAN_ALIGN 
        --match 'g: (ulanid)-[]->(), w: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode) as node'
    """)

In [16]:
import subprocess

values = subprocess.check_output("wc -l < $GETTY_AGREE", shell=True)
values = values.decode("utf-8").strip()
values = int(values) - 1

In [17]:
values / nodes.iloc[0]['node']

1.0015508079384015